In [2]:
import seaborn as sns
import numpy as np
import pandas as pd

### 1. IRIS

#### a. 기초통계량

In [3]:
iris = sns.load_dataset("iris")

In [4]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [5]:
# species로 groupby를 하고, mean과 std만 확인하기
iris.groupby(iris.species).agg(['mean', 'std'])

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa            5.006  0.352490       3.428  0.379064        1.462   
versicolor        5.936  0.516171       2.770  0.313798        4.260   
virginica         6.588  0.635880       2.974  0.322497        5.552   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.173664       0.246  0.105386  
versicolor  0.469911       1.326  0.197753  
virginica   0.551895       2.026  0.274650

#### b.이상치를 제거한 평균과 표준편차

In [6]:
# 이상치 제거 = 해설
# 1. 함수 만들기(시리즈)
def get_new_stat(x):
    q1 = x.quantile(.25)
    q3 = x.quantile(.75)
    iqr = q3-q1
    lower  = q1-1.5*iqr
    higher = q3+1.5*iqr
    x[(x > higher) | (x < lower)] = np.nan #이상치를 np.nan으로 만들기
    outlier = x.value_counts().sum() != 50 #outlier가 50이 아니면 True, 없으면 False
    return outlier, x.mean(), x.std()

In [8]:
# 함수를 통하여 갖고온것이 시리즈가 맞는지 확인
s_sl = iris[iris.species == 'setosa']['sepal_length']
type(s_sl)

pandas.core.series.Series

In [4]:
# 이상치 제거 = 내가한거
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) | (df_in[col_name] < fence_high)]
    return df_out
# sepal_length 이상치제거
re1 = remove_outlier(iris, 'sepal_length')
# sepal_width 이상치제거
re2 = remove_outlier(re1, 'sepal_width')
# petal_length 이상치제거
re3 = remove_outlier(re2, 'petal_length')
# petal_width 이상치제거
re4 = remove_outlier(re3, 'petal_width')

#iris = re4
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
# 4가지 항목의 표준편차
round(iris.std(),2)

sepal_length    0.83
sepal_width     0.44
petal_length    1.77
petal_width     0.76
dtype: float64

In [6]:
# 4가지 항목의 평균
round(iris.mean(),2)

sepal_length    5.84
sepal_width     3.06
petal_length    3.76
petal_width     1.20
dtype: float64

### 2.Titanic

#### a. [category1]

In [10]:
titanic = sns.load_dataset('titanic')

In [11]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
titanic["category1"] = titanic.apply(lambda r: r.sex if r.age>=20 else "child", axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


#### b. ['age']바꾸기

In [13]:
titanic['age'] = round(titanic['age'].fillna(titanic['age'].mean()),2)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [14]:
# a를 다시계산
titanic["category1"] = titanic.apply(lambda r: r.sex if r.age>=20 else "child", axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


#### c. 생존률 구하기

In [23]:
# 성별별 생존률
# alive(문자열)보다는 정수형(survived)로 구하면 좋다.
# 만약에 이후에도 구할시에는 정수형으로 변경하면 보기 더 좋다.
titanic.pivot_table('survived', 'sex')

,survived
sex,
female,0.742038
male,0.188908


In [28]:
titanic[['sex','survived']].groupby(['sex']).mean()
.style.background_gradient(cmap='summer-r')

SyntaxError: invalid syntax (<ipython-input-28-85dbe6969ede>, line 2)

In [21]:
# 선실별 생존률
round(titanic.pivot_table('survived', 'class'),2)

,survived
class,
First,0.63
Second,0.47
Third,0.24


In [22]:
# 출발지별 생존률
round(titanic.pivot_table('survived', 'embark_town'),2)

,survived
embark_town,
Cherbourg,0.55
Queenstown,0.39
Southampton,0.34


In [29]:
titanic.pivot_table('survived', ['sex','class'])

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

#### d. 그룹별 생존률

In [35]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanic['age_cat'] = pd.cut(titanic["age"], bins, labels=labels)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,q_ages,age_cat
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male,청소년,청년
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female,노인,중년
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female,성인,청년
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female,노인,중년
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male,노인,중년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male,성인,청년
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,청소년,미성년자
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,female,성인,청년
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male,성인,청년


In [36]:
titanic.pivot_table('survived', 'age_cat')

,survived
age_cat,
미성년자,0.424242
청년,0.334152
중년,0.423237
장년,0.355932
노년,0.200000


#### e. qcut을 사용한 그룹별 생존률

In [30]:
titanic["q_ages"] = pd.qcut(titanic["age"], 3, labels=['청소년', '성인', '노인'])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,q_ages
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,성인
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,청소년
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,성인
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,성인
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,노인


In [40]:
# gender: 남성이면 1, 여성이면 0
titanic['gender'] = titanic.apply(lambda r: 1 if r.sex == 'male' else 0, axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,q_ages,age_cat,gender
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,성인,청년,1
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,청소년,미성년자,0
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,성인,청년,0
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,성인,청년,1
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,노인,중년,1


In [41]:
titanic.pivot_table('gender', 'q_ages')

,gender
q_ages,
청소년,0.594684
성인,0.680921
노인,0.667832


In [42]:
# 나이 그룹별 남녀 생존률
titanic.pivot_table(['survived', 'gender'], 'q_ages')

,gender,survived
q_ages,,
청소년,0.594684,0.411960
성인,0.680921,0.335526
노인,0.667832,0.405594


In [48]:
titanic[['q_ages','survived','gender']].groupby(['q_ages']).mean()

,survived,gender
q_ages,,
청소년,0.411960,0.594684
성인,0.335526,0.680921
노인,0.405594,0.667832


### mpg

In [76]:
mpg = sns.load_dataset('mpg')

#### a. 배기량 대비 마력

In [77]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [78]:
mpg["hp_per_cc"] = mpg.horsepower/mpg.displacement
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,0.614286
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,0.536082
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,0.622222
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,0.658333
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10,0.689076


#### b. 제조사와 모델을 추출하여 name 컬럼 바꾸기

In [79]:
name_split = mpg['name'].str.split(' ')

manufacturer = name_split.str[0]
mo1 = name_split.str[1]
mo2 = name_split.str[2].fillna('')
model = mo1 + ' ' + mo2

mpg['manufacturer'] = manufacturer
mpg['model'] = model

mpg.drop(['name'], axis='columns', inplace=True)

mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,0.614286,ford,mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,0.536082,vw,pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,0.622222,dodge,rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,0.658333,ford,ranger


#### c. 실린더 갯수별 연비의 평균

In [80]:
mpg.pivot_table('mpg','cylinders')

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


#### d. 생산지별 배기량 대비 마력 평균

In [81]:
mpg[['origin','hp_per_cc']].groupby('origin').mean()

,hp_per_cc
origin,
europe,0.740005
japan,0.789485
usa,0.512063


#### 모델이 5개 이상인 제조사중 연비 평균이 좋은 제조사 5개

In [83]:
mpg.manufacturer.value_counts()

ford             51
chevrolet        43
plymouth         31
amc              28
dodge            28
toyota           25
datsun           23
buick            17
pontiac          16
volkswagen       15
honda            13
mercury          11
mazda            10
oldsmobile       10
peugeot           8
fiat              8
audi              7
volvo             6
chrysler          6
vw                6
renault           5
opel              4
saab              4
subaru            4
chevy             3
cadillac          2
mercedes-benz     2
bmw               2
maxda             2
triumph           1
mercedes          1
nissan            1
hi                1
toyouta           1
vokswagen         1
capri             1
chevroelt         1
Name: manufacturer, dtype: int64

In [85]:
# 모델이 5개인 회사
s = mpg.manufacturer.value_counts()
big = s[s >= 5].index.tolist()
print(big)

['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun', 'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'mazda', 'oldsmobile', 'peugeot', 'fiat', 'audi', 'volvo', 'chrysler', 'vw', 'renault']


In [86]:
mpg[mpg['manufacturer'].isin(big)].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [90]:
mpg[mpg['manufacturer'].isin(big)][['manufacturer','mpg']].groupby('manufacturer').mean()

,mpg
manufacturer,
amc,18.246429
audi,26.714286
buick,19.182353
chevrolet,20.472093
chrysler,17.266667
datsun,31.113043
dodge,22.060714
fiat,28.912500
ford,19.694118


In [88]:
# 위에걸 피봇테이블로
mpg[mpg['manufacturer'].isin(big)][['manufacturer','mpg']].pivot_table('mpg','manufacturer')

,mpg
manufacturer,
amc,18.246429
audi,26.714286
buick,19.182353
chevrolet,20.472093
chrysler,17.266667
datsun,31.113043
dodge,22.060714
fiat,28.912500
ford,19.694118


In [94]:
mpg[mpg['manufacturer'].isin(big)][['manufacturer','mpg']].groupby('manufacturer').mean().sort_values(by='manufacturer', ascending=False)[:5]

,mpg
manufacturer,
vw,39.016667
volvo,21.116667
volkswagen,29.106667
toyota,28.372000
renault,32.880000


In [96]:
x = [1, 2, '해피']
x.append(True)
x

[1, 2, '해피', True]